In [21]:
from smith_waterman.algs import *

In [22]:
method = 'BLOSUM50'

In [39]:
score, align1, align2, align = sw("GCTA", "GCTTTA", "BLOSUM50")

In [40]:
align

'GCT--A'

In [41]:
score

26

In [42]:
align

'GCT--A'

In [43]:
get_score(align1, align2, align, "BLOSUM50")

match G
match C
match T
open -
extend -
match A


19

In [44]:
score

26

In [45]:
scoring = read_scoring_matrix("BLOSUM50")

In [46]:
scoring['G']['A']

0

In [47]:
seq1 = "GCTA" 
seq2 = "GCTTTA"

In [48]:
M, X, Y = initialize_scoring_matrices(seq1, seq2)
scoring = read_scoring_matrix(method)
M, X, Y , PM, PX, PY= fill_matrix(M, X, Y, scoring, seq1, seq2, -10, -1)
SMat, T, score = choose_trace_start(M, X, Y, PM, PX, PY)

In [58]:
Y

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0,  0],
       [ 0,  0,  9, 15,  4,  3,  2]])

In [49]:
SMat

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  8, -3, -2, -2, -2,  0],
       [ 0, -3, 21, -1, -1, -1, -1],
       [ 0, -2, -1, 26, 15, 14,  8],
       [ 0,  0, -1, 10, 26, 15, 19]])

In [50]:
T

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 3, 3, 3, 3, 3, 3],
       [0, 3, 3, 2, 2, 2, 2],
       [0, 3, 2, 3, 2, 2, 2],
       [0, 3, 2, 1, 3, 3, 3]])

In [51]:
score

26

In [52]:
align1, sym, align2, identity = traceback(T, seq1, seq2)

In [53]:
align1

'GCT--A'

In [54]:
sym

'GCT--A'

In [55]:
align2

'GCTTTA'